# Active catheter tracking notebook: test run
To run the full pipeline from raw data download through reconstruction and analysis requires resources that may not be available depending on your setup: about 5 GB storage and about 50 minutes on a single processor (much less time required for a multi-processor system). On cloud notebooks, these resources may not be available, in which case, skip to the analysis section, which uses precomputed outputs from the reconstruction and localization.

The notebook is broken down into:
- Download the raw tracking projections from each experiment
- Reconstruct
- Localize the coils using each algorithm
- Verify the results against preprocessed data
- Analysis: Run the static and dynamic analysis notebooks. Output is shown here

# Download dataset from Zenodo

Test [zenodo_get](https://github.com/dvolgyes/zenodo_get) to download a subset of a dataset, with record #10676292. The "-g" option specifies the pattern we're looking for. Until our dataset is uploaded, testing things with a random dataset:

In [1]:
!zenodo_get -g ROIs.zip 10676292

Title: A Detailed 3D MRI Brain Atlas of the African Lungfish Protopterus annectens
Keywords: Brain, Atlas, 3D, Lungfish, Protopterus, Neuroanatomy, MRI, Magnetic resonance imaging
Publication date: 2024
DOI: 10.5281/zenodo.10676292
Total size: 2.3 MB

Link: https://zenodo.org/records/10676292/files/ROIs.zip   size: 2.3 MB
100% [......................................................] 2342882 / 2342882
Checksum is correct. (8613a410bd6b4fbb134b5685adeea171)
All files have been downloaded.


The above ROIs.zip won't be used, delete it:

In [2]:
!rm ROIs.zip

**Warning** Download of the active tracking raw data requires approximately 5 GB of space. If you prefer, skip ahead to the analysis section which uses the precomputed outputs.

# Run localization algorithms on raw data
**Warning:** this step is very slow and can take an hour. To use precomputed outputs instead, skip to the analysis section.

Test cathy localize:
- run "cathy localize" on raw projection data: this will generate coordinate output in text files from each of the localization algorithms
- ex/ cathy localize -d 6 -p 7 input_dir output_dir
    - runs the algorithm for coils 6 (distal) and 7 (proximal) for the projection files under "input_dir"
    - output_dir will contain subdirectories for each of the algorithms (peak, centroid, centroid_around_peak, png, jpng)
    - each algorithm subdirectory will contain a coordinate text output file for each coil

for this test:
- input_dir: /data/activeTracking-reorg1/data/raw/static/trackTest-13Dec2021-C306-Y0/1/FH512_noDither_gradSpoiled-2021-12-13T13_02_33.756
- output_dir: /data/localize_c306-y0-1-fh

In [3]:
import csv
import pathlib
import os
import multiprocessing as mp
import shutil
import hashlib
from ipywidgets import IntProgress
from IPython.display import display
import numpy as np

In [4]:
note_dir = os.getcwd()
note_dir

'/code'

In [5]:
mp.cpu_count()

24

In [6]:
!cathy localize -d 6 -p 7 /data/activeTracking-reorg1/data/raw/static/trackTest-13Dec2021-C306-Y0/1/FH512_noDither_gradSpoiled-2021-12-13T13_02_33.756 /data/localize_c306-y0-1-fh 

--- -- -- -- -- -- -- -- -- -- --
[0] pre-processing data
[0] processing using peak
[0] processing using centroid
[0] processing using centroid_around_peak
[0] processing using png
[0] processing using jpng


Test the equivalent regular python method, with option to set which algorithms to run:

In [7]:
import cathy.cli as cat
#src = '/data/activeTracking-reorg1/data/raw/static/trackTest-13Dec2021-C306-Y0/1/FH512_noDither_gradSpoiled-2021-12-13T13_02_33.756'
#dst = '/data/loc_c306-y0-1-fh'
#cat.run_localize(src,dst,6,7,1,0,['centroid_around_peak', 'jpng'])

The full list of raw data directories is in the included csv file.

In [8]:
recordings_csv = note_dir + '/data/meta/catheter_raw_recordings.csv'
prepend = '/data/activeTracking-reorg1/data/' # Set this to 'data/' after get from zenodo

The below cell will reconstruct all the raw data and run the localization algorithms using a single cpu: this will be slow. On non-binder instance: Skip this and run the next two cells to distribute the work across cpus & finish faster.

```python
%%time

directory_list = [] # list of preprocessed & processed output directory tuples
doRun = True # True to run cathy localize: ow/ will fill in the directory list but not run cathy

with open(recordings_csv,'r') as csvfile:
    rdr = csv.DictReader(csvfile)
    for row in rdr:
        raw_dir = pathlib.PurePath(row['Input'])
        source = pathlib.PurePath(prepend).joinpath(raw_dir)
        distal = int(row['distal'])
        proximal = int(row['proximal'])
        dest = pathlib.PurePath(prepend).joinpath('processed').joinpath(pathlib.PurePath(*raw_dir.parts[1:]))
        preproc = pathlib.PurePath(prepend).joinpath('preprocessed').joinpath(pathlib.PurePath(*raw_dir.parts[1:]))
        directory_list.append( (preproc,dest))
        print('source: ' + str(source))
        print('dest: ' + str(dest))
        if doRun:
            os.makedirs(dest, exist_ok=True)
            # !cathy localize -d {distal} -p {proximal} {source} {dest}
            cat.run_localize(source, dest, distal, proximal, algos=['centroid_around_peak', 'png', 'jpng'])
```

In [9]:
def call_localize(args):
    assert(len(args[0]) == 4), "Parameter issue: " + str(args)
    src = args[0][0]
    dst = args[0][1]
    dist = args[0][2]
    prox = args[0][3]
    print("run src: " + str(src) + ", dest: " + str(dst) + ", distal: " + str(dist) + ", proximal: " + str(prox))
    return cat.run_localize(src, dst, dist, prox, algos=['centroid_around_peak', 'png', 'jpng'],output_iterations=True)
    

In [10]:
print(os.getenv("CPU_LIMIT"))

None


In [25]:
%%time
# list of preprocessed & processed output directories and coil args
experiment_data_tuples = {'static':[], 'dynamic':[]}

doRun = True # True to run cathy localize: ow/ will fill in the data_tuples but not run localize

#new_dir_list = [] #compare reorg processed directory to repo preprocessed directory (we added png back in)
repo_dir = note_dir + '/data/'

cpu_limit_env = os.getenv("CPU_LIMIT")
if cpu_limit_env is None:
    cpu_count = mp.cpu_count()
else:
    cpu_count = cpu_limit_env
# set up the localize arguments based on the spreadsheet:
with open(recordings_csv,'r') as csvfile:
    rdr = csv.DictReader(csvfile)
    for row in rdr:
        raw_dir = pathlib.PurePath(row['Input'])
        expmt = 'dynamic'
        if 'static' in str(raw_dir):
            expmt = 'static'
        source = pathlib.PurePath(prepend).joinpath(raw_dir)
        distal = int(row['distal'])
        proximal = int(row['proximal'])
        dest = pathlib.PurePath(prepend).joinpath('processed').joinpath(pathlib.PurePath(*raw_dir.parts[1:]))
        experiment_data_tuples[expmt].append( (str(source), dest, distal, proximal))
        preproc = pathlib.PurePath(repo_dir).joinpath('preprocessed').joinpath(pathlib.PurePath(*raw_dir.parts[1:]))
        #new_dir_list.append( (preproc,dest))
        os.makedirs(dest, exist_ok=True)

# data_tuples contains our arguments: we can split the processing across cpus for performance
print("Running localize on " + str(len(experiment_data_tuples['static'])) + " static directories, and " \
     + str(len(experiment_data_tuples['dynamic'])) + " dynamic directories" )
pool = mp.Pool(mp.cpu_count())

iterations_overall = {'png':[],'jpng':[]} # accumulate iterations required for each recording

iterations_xpmt = { 'static':{'png':[],'jpng':[]}, 'dynamic':{'png':[],'jpng':[]} } #separated by experiment type

if doRun:
    for xkey in experiment_data_tuples.keys():
        print("Experiment: " + xkey)
        for result in pool.map( call_localize, [experiment_data_tuples[xkey][i:i+1] for i in range(0,len(experiment_data_tuples[xkey]))] ):
            for key in result.keys():
                iterations_overall[key] = np.concatenate((iterations_overall[key],result[key]))
                iterations_xpmt[xkey][key] = np.concatenate( (iterations_xpmt[xkey][key],result[key]) )


Running localize on 287 static directories, and 6 dynamic directories
run src: /data/activeTracking-reorg1/data/raw/static/trackTest-13Dec2021-C306-Y0/10/FH512_noDither_gradSpoiled-2021-12-13T15_06_28.425, dest: /data/activeTracking-reorg1/data/processed/static/trackTest-13Dec2021-C306-Y0/10/FH512_noDither_gradSpoiled-2021-12-13T15_06_28.425, distal: 6, proximal: 7run src: /data/activeTracking-reorg1/data/raw/static/trackTest-15Dec2021-C231-Y0/13/SRI_Original-2021-12-15T17_26_43.999, dest: /data/activeTracking-reorg1/data/processed/static/trackTest-15Dec2021-C231-Y0/13/SRI_Original-2021-12-15T17_26_43.999, distal: 6, proximal: 7run src: /data/activeTracking-reorg1/data/raw/static/trackTest-13Dec2021-C306-Y0/4/SRI_Original-2021-12-13T13_55_11.879, dest: /data/activeTracking-reorg1/data/processed/static/trackTest-13Dec2021-C306-Y0/4/SRI_Original-2021-12-13T13_55_11.879, distal: 6, proximal: 7run src: /data/activeTracking-reorg1/data/raw/static/trackTest-13Dec2021-C306-Y0/14/SRI_Original-

[0] pre-processing data









[0] pre-processing data
[0] pre-processing data
Experiment: static
[0] processing using centroid_around_peak[0] processing using centroid_around_peak
[0] processing using centroid_around_peak
[0] processing using centroid_around_peak[0] processing using centroid_around_peak[0] processing using centroid_around_peak[0] processing using centroid_around_peak

[0] processing using centroid_around_peak[0] processing using centroid_around_peak[0] processing using centroid_around_peak

[0] processing using centroid_around_peak


[0] processing using centroid_around_peak

[0] processing using centroid_around_peak
[0] processing using centroid_around_peak

[0] processing using centroid_around_peak[0] processing using centroid_around_peak
[0] processing using centroid_around_peak[0] processing using centroid_around_peak


[0] processing using centroid_around_peak
[0] processing using centroid_around_peak
[0] processing using centroid_around_peak
[0] processing usi

[0] processing using png
[0] processing using jpng
[0] processing using jpng
[0] processing using jpng
run src: /data/activeTracking-reorg1/data/raw/static/trackTest-15Dec2021-C231-Y0/7/FH512_noDither_gradSpoiled-2021-12-15T15_54_04.360, dest: /data/activeTracking-reorg1/data/processed/static/trackTest-15Dec2021-C231-Y0/7/FH512_noDither_gradSpoiled-2021-12-15T15_54_04.360, distal: 6, proximal: 7
--- -- -- -- -- -- -- -- -- -- --
[0] pre-processing data
[0] processing using jpng
[0] processing using centroid_around_peak
[0] processing using jpng
[0] processing using png
run src: /data/activeTracking-reorg1/data/raw/static/trackTest-17Aug2021-C222-Y0/13/SRI_Original-2021-08-17T16_14_41.234, dest: /data/activeTracking-reorg1/data/processed/static/trackTest-17Aug2021-C222-Y0/13/SRI_Original-2021-08-17T16_14_41.234, distal: 4, proximal: 5
--- -- -- -- -- -- -- -- -- -- --
[0] pre-processing data
run src: /data/activeTracking-reorg1/data/raw/static/trackTest-13Dec2021-C306-Y0/16/SRI_Original

run src: /data/activeTracking-reorg1/data/raw/static/trackTest-17Aug2021-C222-Y0/14/FH512_noDither_gradSpoiled-2021-08-17T16_27_41.342, dest: /data/activeTracking-reorg1/data/processed/static/trackTest-17Aug2021-C222-Y0/14/FH512_noDither_gradSpoiled-2021-08-17T16_27_41.342, distal: 4, proximal: 5
--- -- -- -- -- -- -- -- -- -- --
[0] pre-processing data
[0] processing using png
[0] processing using jpng
[0] processing using centroid_around_peak
[0] processing using centroid_around_peak
[0] processing using jpng
[0] processing using centroid_around_peak
[0] processing using jpng
[0] processing using jpng
[0] processing using png
[0] processing using png
run src: /data/activeTracking-reorg1/data/raw/static/trackTest-15Dec2021-C231-Y0/4/SRI_Original-2021-12-15T15_18_39.502, dest: /data/activeTracking-reorg1/data/processed/static/trackTest-15Dec2021-C231-Y0/4/SRI_Original-2021-12-15T15_18_39.502, distal: 6, proximal: 7
--- -- -- -- -- -- -- -- -- -- --
[0] pre-processing data
[0] processin

--- -- -- -- -- -- -- -- -- -- --[0] pre-processing data

[0] pre-processing data
run src: /data/activeTracking-reorg1/data/raw/static/trackTest-17Aug2021-C222-Y0/10/SRI_Original-2021-08-17T14_50_18.200, dest: /data/activeTracking-reorg1/data/processed/static/trackTest-17Aug2021-C222-Y0/10/SRI_Original-2021-08-17T14_50_18.200, distal: 4, proximal: 5
--- -- -- -- -- -- -- -- -- -- --
[0] pre-processing data
run src: /data/activeTracking-reorg1/data/raw/static/trackTest-17Aug2021-C222-Y0/1/SRI_Original-2021-08-17T19_12_32.999, dest: /data/activeTracking-reorg1/data/processed/static/trackTest-17Aug2021-C222-Y0/1/SRI_Original-2021-08-17T19_12_32.999, distal: 4, proximal: 5
--- -- -- -- -- -- -- -- -- -- --
[0] pre-processing data
[0] processing using centroid_around_peak
[0] processing using centroid_around_peak
[0] processing using centroid_around_peak
[0] processing using centroid_around_peak
[0] processing using jpng
[0] processing using png
[0] processing using png
[0] processing using

...

[0] processing using centroid_around_peak
[0] processing using png
[0] processing using centroid_around_peak
[0] processing using jpng
[0] processing using png
[0] processing using png
run src: /data/activeTracking-reorg1/data/raw/static/trackTest-9Jan2022-C231-Y2/12/SRI_Original-2022-01-09T17_09_14.726, dest: /data/activeTracking-reorg1/data/processed/static/trackTest-9Jan2022-C231-Y2/12/SRI_Original-2022-01-09T17_09_14.726, distal: 6, proximal: 7
--- -- -- -- -- -- -- -- -- -- --
[0] pre-processing data
run src: /data/activeTracking-reorg1/data/raw/static/trackTest-8Nov2021-C222-Y1/1/FH512_noDither_gradSpoiled-2021-11-08T15_52_01.435, dest: /data/activeTracking-reorg1/data/processed/static/trackTest-8Nov2021-C222-Y1/1/FH512_noDither_gradSpoiled-2021-11-08T15_52_01.435, distal: 6, proximal: 7
--- -- -- -- -- -- -- -- -- -- --
[0] pre-processing data
[0] processing using centroid_around_peak
run src: /data/activeTracking-reorg1/data/raw/static/trackTest-7Jan2022-C306-Y2/16/SRI_Original

run src: /data/activeTracking-reorg1/data/raw/static/trackTest-9Jan2022-C231-Y2/14/SRI_Original-2022-01-09T17_31_20.796, dest: /data/activeTracking-reorg1/data/processed/static/trackTest-9Jan2022-C231-Y2/14/SRI_Original-2022-01-09T17_31_20.796, distal: 6, proximal: 7
--- -- -- -- -- -- -- -- -- -- --
[0] pre-processing data
[0] processing using jpng
[0] processing using centroid_around_peak
[0] processing using jpng[0] processing using jpng

run src: /data/activeTracking-reorg1/data/raw/static/trackTest-8Nov2021-C222-Y1/7/SRI_Original-2021-11-08T17_41_29.242, dest: /data/activeTracking-reorg1/data/processed/static/trackTest-8Nov2021-C222-Y1/7/SRI_Original-2021-11-08T17_41_29.242, distal: 6, proximal: 7
--- -- -- -- -- -- -- -- -- -- --
[0] pre-processing data
[0] processing using png
run src: /data/activeTracking-reorg1/data/raw/static/trackTest-9Jan2022-C231-Y2/5/SRI_Original-2022-01-09T15_46_01.531, dest: /data/activeTracking-reorg1/data/processed/static/trackTest-9Jan2022-C231-Y2/5/

run src: /data/activeTracking-reorg1/data/raw/static/trackTest-9Nov2021-C222-Y2/3/FH512_noDither_gradSpoiled-2021-11-09T15_41_21.334, dest: /data/activeTracking-reorg1/data/processed/static/trackTest-9Nov2021-C222-Y2/3/FH512_noDither_gradSpoiled-2021-11-09T15_41_21.334, distal: 6, proximal: 7
--- -- -- -- -- -- -- -- -- -- --
[0] pre-processing data
[0] processing using png
[0] processing using centroid_around_peak
[0] processing using centroid_around_peak
[0] processing using png
run src: /data/activeTracking-reorg1/data/raw/static/trackTest-9Jan2022-C231-Y2/15/FH512_noDither_gradSpoiled-2022-01-09T17_42_47.721, dest: /data/activeTracking-reorg1/data/processed/static/trackTest-9Jan2022-C231-Y2/15/FH512_noDither_gradSpoiled-2022-01-09T17_42_47.721, distal: 6, proximal: 7
--- -- -- -- -- -- -- -- -- -- --
[0] pre-processing data
[0] processing using png
run src: /data/activeTracking-reorg1/data/raw/static/trackTest-9Jan2022-C231-Y2/13/SRI_Original-2022-01-09T17_20_58.450, dest: /data/ac

run src: /data/activeTracking-reorg1/data/raw/static/trackTest-9Jan2022-C231-Y2/10/SRI_Original-2022-01-09T16_43_30.413, dest: /data/activeTracking-reorg1/data/processed/static/trackTest-9Jan2022-C231-Y2/10/SRI_Original-2022-01-09T16_43_30.413, distal: 6, proximal: 7
--- -- -- -- -- -- -- -- -- -- --
[0] pre-processing data
[0] processing using centroid_around_peak
[0] processing using png
[0] processing using centroid_around_peak
[0] processing using jpng
[0] processing using jpng
[0] processing using jpng
[0] processing using png
run src: /data/activeTracking-reorg1/data/raw/static/trackTest-9Nov2021-C222-Y2/9/FH512_noDither_gradSpoiled-2021-11-09T17_04_47.828, dest: /data/activeTracking-reorg1/data/processed/static/trackTest-9Nov2021-C222-Y2/9/FH512_noDither_gradSpoiled-2021-11-09T17_04_47.828, distal: 6, proximal: 7
--- -- -- -- -- -- -- -- -- -- --
[0] pre-processing data
[0] processing using png
[0] processing using centroid_around_peak
run src: /data/activeTracking-reorg1/data/r

run src: /data/activeTracking-reorg1/data/raw/static/trackTest-9Nov2021-C222-Y2/1/FH512_noDither_gradSpoiled-2021-11-09T15_12_58.486, dest: /data/activeTracking-reorg1/data/processed/static/trackTest-9Nov2021-C222-Y2/1/FH512_noDither_gradSpoiled-2021-11-09T15_12_58.486, distal: 6, proximal: 7
--- -- -- -- -- -- -- -- -- -- --
[0] pre-processing data
[0] processing using png
[0] processing using centroid_around_peak
[0] processing using jpng
[0] processing using png
[0] processing using jpng
[0] processing using jpng
run src: /data/activeTracking-reorg1/data/raw/static/trackTest-9Nov2021-C222-Y2/7/FH512_noDither_gradSpoiled-2021-11-09T16_42_21.191, dest: /data/activeTracking-reorg1/data/processed/static/trackTest-9Nov2021-C222-Y2/7/FH512_noDither_gradSpoiled-2021-11-09T16_42_21.191, distal: 6, proximal: 7
--- -- -- -- -- -- -- -- -- -- --
[0] pre-processing data
[0] processing using jpng
[0] processing using centroid_around_peak
[0] processing using jpng
[0] processing using png
[0] pro

In [26]:
len(iterations_overall['png'])

84789

In [27]:
len(iterations_overall['jpng'])

84789

In [28]:
len(iterations_xpmt['static']['png'])

75267

In [29]:
len(iterations_xpmt['static']['jpng'])

75267

In [30]:
len(iterations_xpmt['dynamic']['png'])

9522

In [31]:
len(iterations_xpmt['dynamic']['jpng'])

9522

In [32]:
iterations_overall.keys()

dict_keys(['png', 'jpng'])

In [33]:
for key in iterations_overall.keys():
    print(f"{key}: mean iterations {np.mean(iterations_overall[key])}\t, median {np.median(iterations_overall[key])},\
    min {np.min(iterations_overall[key])}, max {np.max(iterations_overall[key])}")

png: mean iterations 23.21567656181816	, median 22.0,    min 11.0, max 32.0
jpng: mean iterations 25.053497505572658	, median 22.0,    min 11.0, max 256.0


In [34]:
for xkey in iterations_xpmt:
    print(f"Experiment set: {xkey}")
    for key in iterations_xpmt[xkey]:
        print(f"{key}: mean iterations {np.mean(iterations_xpmt[xkey][key])}\t, \
        median {np.median(iterations_xpmt[xkey][key])},\
        min {np.min(iterations_xpmt[xkey][key])}, max {np.max(iterations_xpmt[xkey][key])}")

Experiment set: static
png: mean iterations 22.245021058365552	,         median 21.0,        min 11.0, max 32.0
jpng: mean iterations 22.87965509453014	,         median 21.0,        min 11.0, max 256.0
Experiment set: dynamic
png: mean iterations 30.888258769166143	,         median 32.0,        min 14.0, max 32.0
jpng: mean iterations 42.23671497584541	,         median 36.0,        min 14.0, max 256.0


In [ ]:
import os
print(os.cpu_count())

In [ ]:
!python --version

# Copy ground truth data to processed directory
Ground truth files are expected in the processed directory for later analysis. These should be copied from the raw subdirectories to the corresponding processed subdirectories. A list of these files is in meta/gt_files.txt

In [ ]:
gtf = open(note_dir + '/data/meta/gt_files.txt','r')
gt_list = gtf.readlines()
for gt_file in gt_list:
    gt_path = pathlib.PurePath(gt_file.rstrip())
    src = pathlib.PurePath(prepend).joinpath(gt_path)
    dest = pathlib.PurePath(prepend).joinpath('processed').joinpath(pathlib.PurePath(*gt_path.parts[1:]))
    dest_dir = pathlib.PurePath(*dest.parts[:-1])
    print('dest dir: ' + str(dest_dir))
    os.makedirs(dest_dir, exist_ok=True)
    print('cp ' + str(src) + ' ' + str(dest))
    shutil.copyfile(src,dest)
    

## Verify processed outputs
Check the processed outputs against the preprocessed. Initial check below will compare files in detail to ensure this method works - the original data only stored an SNR of 0 so we need to ignore this field until we can verify the rest of the data.

```python
import catheter_utils.cathcoords
import glob
import numpy as np

algos = ['centroid_around_peak','png','jpng']

error_count = 0
EPS = 0.001

def distance(c1, c2):
    return np.linalg.norm(c2-c1)

proc_file_list = [] # This will contain a list of the output files
for src_test, dst_test in new_dir_list:
    for algo in algos:
        src_dir = pathlib.PurePath(src_test).joinpath(algo)
        dst_dir = pathlib.PurePath(dst_test).joinpath(algo)
        src_files = glob.glob(str(src_dir)+'/cathcoords*.txt')
        for src in src_files:
            srcpath = pathlib.PurePath(src)
            fname = srcpath.parts[-1]
            dstpath = dst_dir.joinpath(fname)
            if (os.path.isfile(dstpath)):
                cc_src = catheter_utils.cathcoords.read_file(srcpath)
                #print("cc_src: " + str(cc_src))
                cc_dst = catheter_utils.cathcoords.read_file(dstpath)
                proc_file_list.append(dstpath)
                # Check each xyz coordinate
                src_coords = cc_src.coords
                dst_coords = cc_dst.coords
                dists = list(map(distance, src_coords, dst_coords))
                times_equal = np.array_equal(cc_src.times,cc_dst.times)
                trigs_equal = np.array_equal(cc_src.trigs,cc_dst.trigs)
                resps_equal = np.array_equal(cc_src.resps,cc_dst.resps)
                if (any(i > EPS for i in dists) or not times_equal or not trigs_equal or not resps_equal):
                    print("Mismatch in: " + str(src))
                    error_count += 1
                else:
                    print(".", end = "")
            else:
                print("Missing: " + str(dstpath))
                error_count += 1
print("\nErrors: " + str(error_count))
```

# Hash of files
The processed files match with the old dataset (except for SNR, which was not implemented in the original code). Use the new processed files as the canonical dataset. Create a md5sums.txt file for all the cathcoords outputs.

In [ ]:
def generate_md5(filename, chunk_size=4096):
    md5 = hashlib.md5()
    with open(filename, "rb") as f:
        # digest = hashlib.file_digest(f, digest) Python >= 3.11 reqd!
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            md5.update(chunk)
        return md5.hexdigest()

## This was already run to create the md5sums file

```python
proc_md5_sum = prepend + 'preprocessed/proc_md5sums.txt'
with open(proc_md5_sum, "w") as f_out:
    for proc_file in proc_file_list:
        md5_string = generate_md5(str(proc_file))
        f_out.write("{} : {}\n".format(str(proc_file), md5_string))
```

# Verify processed files against md5sums

In [ ]:
preproc_md5_sum = note_dir + '/data/meta/proc_md5sums.txt'
error_count = 0

with open(preproc_md5_sum, "r") as f:
    reader = csv.reader(f, delimiter=':')
    records = sum(1 for row in reader)
    f.seek(0)
    bar = IntProgress(min=0, max=records,description='Verifying:')
    display(bar)
    for row in reader:
        proc_file = row[0].strip()
        proc_digest = row[1].strip()
        if (not os.path.isfile(proc_file)):
            print('Coordinate file ' + proc_file + ' missing')
            error_count += 1
        else:
            digest = generate_md5(proc_file)
            bar.value += 1
            if (digest != proc_digest):
                print('Coordinate file ' + proc_file + ' mismatch')
                error_count += 1

print('\nErrors: ' + str(error_count))

In [ ]:
cpu_limit = os.environ.get("CPU_LIMIT")

In [ ]:
if cpu_limit is None:
    cpu_limit = os.cpu_count()

In [ ]:
cpu_limit

In [ ]:
def get_cpu_quota_within_docker():
    cpu_cores = None
    cfs_period = pathlib.Path("/sys/fs/cgroup/cpu/cpu.cfs_period_us")
    cfs_quota = pathlib.Path("/sys/fs/cgroup/cpu/cpu.cfs_quota_us")

    if cfs_period.exists() and cfs_quota.exists():
        with cfs_period.open('rb') as p, cfs_quota.open('rb') as q:
            p, q = int(p.read()), int(q.read())
            # get the cores allocated by dividing the quota
            # in microseconds by the period in microseconds
            cpu_cores = math.ceil(q / p) if q > 0 and p > 0 else None
    return cpu_cores

In [ ]:
cpu_cores = get_cpu_quota_within_docker()

In [ ]:
cpu_cores

In [ ]:
!cat /sys/fs/cgroup/cpu/cpu.cfs_quota_us

In [ ]:
!cat /sys/fs/cgroup/cpu/cpu.cfs_period_us

# Analysis
The notebooks are in subdirectories. We call each notebook in turn below.
## Static Tracking

In [ ]:
%cd Static_Tracking
%run static_tracking_heatmaps_Y0.ipynb

In [ ]:
%run static_tracking_heatmaps_Y1.ipynb

In [ ]:
%run static_tracking_heatmaps_Y2.ipynb